In [1]:
from simulator_interface import open_session, close_session
import numpy as np
simulator, epuck1, epuck2 = open_session(n_epucks=2)

In [2]:
epuck1.species = "selfish"
epuck2.species = "altruistic"

In [3]:
def foraging_drive(robot): 
    if robot.has_eaten():
        robot.energy_level += 0.2  # if the robot has eaten a sphere, increase its energy level by 0.2
    else:
        robot.energy_level -= 0.01  # otherwise (nothing eaten), decrease the energy level by 0.01
    # The line below bounds the value of the energy level between 0 and 1
    robot.energy_level = min(1., max(robot.energy_level, 0.))

'''
Altruistic routine determined by _w attribute
Find another robot with lower energy ==> _w increases ==> 
==> altruistic behavior increases (see foraging and away_from_spheres behaviors) 
'''
def altruistic_drive(robot):
    (left, right), (epuck_left, epuck_right) = robot.prox_activations(tracked_objects=["ePuck"], return_epucks=True)
    left_energy, right_energy = robot.sensed_epuck_attributes(epuck_left, epuck_right, "energy_level", default_value=0)
    
    not_dead = left_energy > 0. or right_energy > 0.  # some ePuck has been detected and at least one of 
                                                      # them is not dead
    lower_energy = left_energy < robot.energy_level or right_energy < robot.energy_level  # some of the detected 
                                                                                          # ePucks have less energy 
                                                                                          # than me
    if(not_dead and lower_energy):
        if(min(left_energy, right_energy) != 0): 
            low_energy = min(left_energy, right_energy)
        else: 
            low_energy = max(left_energy, right_energy)  # avoid value 0
        robot._w = (np.sqrt(1/low_energy) - 1)/2         # between 0 and 1
    else: 
        robot._w = 0
    
    
def foraging(robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    if(robot.species == "selfish"): 
        left_activation = right
        right_activation = left
        w = 2
    else:
        left_activation = max(right - robot._w, 0.1)
        right_activation = max(left - robot._w, 0.1)
        w = 2 - robot.energy_level - robot._w  # if ePuck is altruistic, foraging behavior decreases when
                                               # _w value increases
    return left_activation, right_activation, w


def away_from_spheres(robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    left_activation = left
    right_activation = right
    if(robot.species == "selfish"): w = 0
    else: w = robot.energy_level + robot._w  # if ePuck is altruistic, away_from_spheres behavior increases
                                             # when _w value increases
    return left_activation, right_activation, w

    
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=["20cm", "Tree", "Cup"])
    return 1 - right, 1 - left

In [4]:
simulator.start_sphere_apparition(period=10)
for e in simulator.robots: 
    e.detach_all_behaviors()
    e.attach_routine(foraging_drive, freq=1)
    e.energy_level = 0.5
    e.start_routine(foraging_drive)
    e.attach_behavior(obstacle_avoidance, freq=10)
    e.attach_behavior(foraging, freq=10)
    if e.species == "altruistic":
        e.attach_routine(altruistic_drive, freq=1)
        e._w = 0.0
        e.start_routine(altruistic_drive)
    e.start_all_behaviors()

Routine sphere_apparition started
Routine eating started
Routine foraging_drive started
Behavior obstacle_avoidance started
Behavior foraging started
Routine foraging_drive started
Routine altruistic_drive started
Behavior obstacle_avoidance started
Behavior foraging started


In [5]:
simulator.stop_sphere_apparition()
for e in simulator.robots: 
    e.detach_all_behaviors()

Routine sphere_apparition stopped
Routine eating stopped
